In [1]:
%load_ext lab_black

In [2]:
import selenium
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from time import sleep
from datetime import datetime

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

import requests
import numpy as np

import time
import re

from tqdm import tqdm
import pickle

import tempfile
from collections import defaultdict

import logging

logging.basicConfig(level=logging.INFO, filename="../logs/scrape.log")

In [3]:
companies = pd.read_csv(
    "../data/intermediate/companies_filling_minimal.csv", index_col=0
)
companies["most_recent_filling"] = pd.to_datetime(companies["most_recent_filling"])
companies_more_2006 = companies[companies["most_recent_filling"].dt.year > 2006]
companies_more_2008 = companies[companies["most_recent_filling"].dt.year > 2008]

ciks = list(companies_more_2008["CIK"])
len(ciks)

4625

In [4]:
headers = {"User-Agent": "Anselme F.E. Borgeaud (aborgeaud@gmail.com)"}

In [5]:
def is_balance_sheet(df):
    ncol = len(df.columns)
    has_assets = df.iloc[:, 0].astype("string").str.contains(r"Assets").sum() > 0
    has_cash_equivs = []
    has_cashs = []
    has_total_liabs = []
    has_equities = []
    has_debts = []
    for icol in range(min(ncol, 3)):
        has_cash_equivs.append(
            df.iloc[:, icol]
            .astype("string")
            .str.contains(r"[cC]ash.*[eE]quivalent.*")
            .sum()
            > 0
        )
        has_cashs.append(
            df.iloc[:, icol].astype("string").str.contains(r"[cC]ash").sum() > 0
        )
        has_total_liabs.append(
            df.iloc[:, icol].astype("string").str.contains(r"Total.*liabilities").sum()
            > 0
        )
        has_debts.append(
            df.iloc[:, icol].astype("string").str.contains(r"Long-term debt").sum() > 0
        )
        has_equities.append(
            df.iloc[:, icol]
            .astype("string")
            .str.contains(r"Total stockholder.*equity")
            .sum()
            > 0
        )

    has_cash = any(has_cash_equivs) or any(has_cashs)
    has_liab = any(has_total_liabs)
    has_debt = any(has_debts)
    has_equity = any(has_equities)

    return has_cash and has_equity and has_assets and (has_debt or has_liab)


def find_index_col(df):
    index_col = ""
    i_index_col = 0
    for i, col in enumerate(df.columns):
        if df.loc[:, col].astype("str").str.contains("Assets").any():
            index_col = col
            i_index_col = i
    return i_index_col, index_col


def is_finite(x):
    try:
        f = float(x)
        return np.isfinite(f)
    except:
        return False


def process_balance(df):
    i, index_col = find_index_col(df)
    arr = df.to_numpy()[:, i:]
    mask = [type(x) == str for x in arr[:, 0]]
    arr = arr[mask, :]
    data = np.apply_along_axis(lambda x: get_value(x), 1, arr)
    index = arr[:, 0]
    df = pd.DataFrame({"value": data}, index=index)
    return df


def is_balance_sheet_2(table):
    content = table.text.lower()
    has_assets = (len(re.findall(r"current.*assets", content, re.DOTALL)) > 0) or (
        len(re.findall(r"total.*assets", content, re.DOTALL)) > 0
    )
    has_liab = (len(re.findall(r"total.*liabilities", content, re.DOTALL)) > 0) or (
        len(re.findall(r"current.*liabilities", content, re.DOTALL)) > 0
    )
    has_cash = "cash" in table.text.lower()
    has_equity = "equity" in table.text.lower()
    has_asset = "assets" in table.text.lower()
    return has_asset and has_assets and has_liab and has_cash and has_equity


def get_balance_table_2(soup):
    tables = soup.findAll("table")
    balance_tables = [t for t in tables if is_balance_sheet_2(t)]
    if len(balance_tables) == 0:
        return None
    i_longest = 0
    longest = 0
    for i, table in enumerate(balance_tables):
        if len(table) > longest:
            i_longest = i
            longest = len(table)
    balance_table = balance_tables[i_longest]
    dfs = pd.read_html(balance_table.prettify(), flavor="bs4")
    if len(dfs) == 1:
        return dfs[0]


def get_value(x):
    vals = [v for v in x if is_finite(v)]
    try:
        val = float(vals[0]) * 1000
    except:
        return None
    return val


def get_date(soup):
    ps = soup.findAll("p")
    bs = soup.findAll("b")
    texts = [
        p.get_text(strip=True).replace(u"\xa0", u" ")
        for p in ps + bs
        if "fiscal year ended" in p.text
    ]
    if len(texts) > 0:
        match = re.findall(r"ended .*[0-9]{4}", texts[0])
        if len(match) == 1:
            date_str = match[0].replace("ended ", "")
            try:
                date = datetime.strptime(date_str, "%B %d, %Y")
                return date
            except:
                return None


def get_balance_table(soup):
    bs = soup.findAll("b")
    b_balance = [b for b in bs if "consolidated balance sheet" in b.text.lower()][0]
    parent = b_balance.find_parent("div")
    i = 0
    table = None
    while (table is None) and (i < 4):
        parent = parent.find_next_sibling("div")
        table = parent.find("table")
        i += 1

    try:
        return pd.read_html(table.prettify(), flavor="bs4")[0]
    except:
        return None


def find_value(row_elem):
    def is_number(s):
        return len(re.findall(r"[0-9]+", s)) > 0

    elem = row_elem.find_next_sibling("td")
    if elem is None:
        return None
    i = 0
    while not is_number(elem.text) and i < 4:
        elem = elem.find_next_sibling("td")
    if is_number(elem.text):
        try:
            return float(elem.text.replace(",", ""))
        except ValueError:
            return None


def find_value_in_table(soup, key: str) -> float:
    key_elem = None
    key_text = key.strip().replace("\n", "_").replace(" ", "_").lower()
    for e in soup.findAll("td"):
        text = e.text.strip().replace("\n", "_").replace(" ", "_").lower()
        if key_text == text:
            key_elem = e
    if key_elem:
        return find_value(key_elem)

In [6]:
entries = ["total_current_assets", "total_current_liabilities"]


def run(ciks: list):
    driver = webdriver.Chrome()
    time.sleep(8)

    report_urls = []
    data_dict = defaultdict(list)
    data_df = None

    for i, cik in enumerate(tqdm(ciks)):
        if (i > 0) and (i % 30 == 0):
            timestamp = time.time_ns()
            file_report = f"../data/intermediate/report_urls{timestamp}.pickle"
            with open(file_report, "wb") as f:
                pickle.dump(report_urls, f)
            data_df = pd.DataFrame(data_dict)
            data_df.to_csv(f"../data/intermediate/scraped_financials{timestamp}.csv")

        url = f"https://www.sec.gov/edgar/browse/?CIK={cik}"
        try:
            driver.get(url)
        except:
            logging.info(f"{url} Did not get page")
            continue

        time.sleep(4)

        try:
            search_elem = driver.find_element_by_xpath(
                '//input[@placeholder="Search table"]'
            )
            search_elem.send_keys("10-K ")  # blank space to avoid 10-K/A (amendments)
        except:
            logging.info(f"{url} Did not input 10-K")
            continue

        link_elems = driver.find_elements_by_class_name("document-link")
        annual_report_elems = [e for e in link_elems if "Annual report" in e.text]
        annual_report_pages = []

        for annual_report_elem in annual_report_elems:
            try:
                resp = requests.get(
                    annual_report_elem.get_property("href"), headers=headers, timeout=5
                )
                annual_report_pages.append(resp)
            except:
                href_elem = a.get_property("href")
                logging.info(f"{href_elem} Did not fetch report page")
                pass

        report_urls.extend([a.get_property("href") for a in annual_report_elems])

        for i, page in enumerate(annual_report_pages):
            soup = BeautifulSoup(page.content)

            try:
                date_str = driver.find_elements_by_xpath(
                    '//a[@data-index="reportDate"]'
                )[i].text
                date = datetime.fromisoformat(date_str)
            except:
                logging.info(f"{page.url} Did not read date")
                continue

            tmp_dict = None
            try:
                tmp_dict = dict()
                for entry in entries:
                    tmp_dict[entry] = find_value_in_table(soup, entry)
            except:
                logging.info(f"{page.url} Did not read html")
                continue
            if (tmp_dict.keys() - set(entries)) == set():
                for entry, value in tmp_dict.items():
                    data_dict[entry].append(value)
                data_dict["CIK"].append(cik)
                data_dict["date_filled"].append(date)
                data_dict["url"].append(page.url)

    data_df = pd.DataFrame(data_dict)
    data_df.to_csv("../data/intermediate/scraped_financials.csv")

    with open("../data/intermediate/report_urls.pickle", "wb") as f:
        pickle.dump(report_urls, f)

In [7]:
n = len(ciks)
print(n)
splits = [n // 3 * i for i in range(4)]
splits[-1] = n
splits

4625


[0, 1541, 3082, 4625]

In [8]:
run(ciks[splits[0] : splits[1]])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1541/1541 [2:24:41<00:00,  5.63s/it]
